In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os

In [ ]:
def load_and_preprocess_data(file_path):
  
   
    
        
    df = pd.read_csv(file_path)
        
       
    df = df[df['timestamp'] != 'timestamp']
        
       
    df['timestamp'] = pd.to_datetime(df['timestamp'], 
                                          format='mixed', 
                                          errors='coerce')
        
       
    df.dropna(subset=['timestamp'], inplace=True)
        
        
    numeric_columns = ['cpu_temperature', 'cpu_usage', 'cpu_load', 
                           'memory_usage', 'battery_level', 'cpu_power']
        
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
       
    df.dropna(inplace=True)
        
        
    df.set_index('timestamp', inplace=True)
        
    return df
    
   

In [ ]:
def detect_anomalies(series, method='zscore', threshold=3):
    
    if method == 'zscore':
       
        z_scores = np.abs(stats.zscore(series))
        return z_scores > threshold
    
    elif method == 'iqr':
        
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        return (series < lower_bound) | (series > upper_bound)

In [ ]:
def plot_anomalies(df, columns, output_dir):
   
    
    plt.rcParams['agg.path.chunksize'] = 10000
    plt.rcParams['path.simplify_threshold'] = 1.0
    
    
    os.makedirs(output_dir, exist_ok=True)
    
   
    output_file = os.path.join(output_dir, 'system_metrics_anomalies.png')
    
    plt.figure(figsize=(20, 15))
    
    for i, col in enumerate(columns, 1):
        plt.subplot(3, 2, i)
        
       
        anomalies_zscore = detect_anomalies(df[col], method='zscore')
        anomalies_iqr = detect_anomalies(df[col], method='iqr')
        
       
        combined_anomalies = anomalies_zscore | anomalies_iqr
        
        
        plt.plot(df.index, df[col], label=col, alpha=0.7)
        plt.scatter(df.index[combined_anomalies], 
                    df.loc[combined_anomalies, col], 
                    color='red', label='Anomalies', s=10)
        
        plt.title(f'{col} Anomaly Detection')
        plt.xlabel('Timestamp')
        plt.ylabel('Value')
        plt.legend(loc='upper right')
        plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig(output_file, dpi=300)
    plt.close()
    
    print(f"Anomaly detection plot saved to: {output_file}")

In [ ]:
import os

def main(file_path):
    
    
    output_dir = os.getcwd()
    
    
    df = load_and_preprocess_data(file_path)
    
   
    columns_to_analyze = [
        'cpu_temperature', 
        'cpu_usage', 
        'cpu_load', 
        'memory_usage', 
        'cpu_power'
    ]
    
   
    plot_anomalies(df, columns_to_analyze, output_dir)
    
    print("\nAnomaly Detection Summary:")
    for col in columns_to_analyze:
        anomalies_zscore = detect_anomalies(df[col], method='zscore')
        anomalies_iqr = detect_anomalies(df[col], method='iqr')
        combined_anomalies = anomalies_zscore | anomalies_iqr
        
        print(f"{col}:")
        print(f"  Total anomalies detected: {combined_anomalies.sum()}")
        print(f"  Percentage of anomalies: {combined_anomalies.sum() / len(df) * 100:.2f}%")
        print()


main(r"C:\syntheticdata\TSADSD-main\hardware_monitor_data_final2.csv")

Anomaly detection plot saved to: c:\syntheticdata\system_metrics_anomalies.png

Anomaly Detection Summary:
cpu_temperature:
  Total anomalies detected: 778627
  Percentage of anomalies: 3.22%

cpu_usage:
  Total anomalies detected: 3147881
  Percentage of anomalies: 13.00%

cpu_load:
  Total anomalies detected: 1759469
  Percentage of anomalies: 7.27%

memory_usage:
  Total anomalies detected: 371831
  Percentage of anomalies: 1.54%

cpu_power:
  Total anomalies detected: 1245702
  Percentage of anomalies: 5.14%

